In [3]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


### Load model

In [8]:
import xgboost

model = xgboost.XGBRegressor()
model.load_model("xgboost-model-pytorch")

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/xgboost/sklearn.py:741: UserWarning: Loading a native XGBoost model with Scikit-Learn interface.
  warnings.warn(


### Load test dataset

In [9]:
import pandas as pd

In [10]:
pd.options.display.max_rows=20
pd.options.display.max_columns=10

In [11]:
df = pd.read_csv('../dataset/test.csv')

In [12]:
df.head()

,fraud,vehicle_claim,total_claim_amount,customer_age,months_as_customer,...,authorities_contacted_Fire,authorities_contacted_None,authorities_contacted_Police,police_report_available_No,police_report_available_Yes
0,0,23901.432507,36201.432507,56,118,...,0,0,1,0,1
1,0,29095.295736,73695.295736,36,71,...,0,0,1,0,1
2,0,11232.831826,25032.831826,24,11,...,0,1,0,1,0
3,0,11125.579356,25125.579356,23,65,...,0,1,0,1,0
4,0,4529.773913,22329.773913,37,142,...,0,0,1,1,0


In [14]:
df.groupby('fraud').size()

fraud
0    967
1     33
dtype: int64

In [15]:
y_test = df.iloc[:, 0].astype('int')    
df.drop(df.columns[0], axis=1, inplace=True)

In [16]:
X_test = df.values

In [19]:
predictions_prob = model.predict(X_test)

#### if predictions_prob is greater than 0.5, it is 1 as a fruad, otherwise it is 0 as a non-fraud

In [20]:
threshold = 0.5
predictions = [1 if e >= 0.5 else 0 for e in predictions_prob ] 

In [36]:
from sklearn.metrics import classification_report

print(classification_report(y_true=y_test, y_pred = predictions))

              precision    recall  f1-score   support

           0       0.99      0.72      0.83       967
           1       0.09      0.79      0.16        33

    accuracy                           0.72      1000
   macro avg       0.54      0.75      0.50      1000
weighted avg       0.96      0.72      0.81      1000



In [35]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true= y_test, y_pred= predictions)    
print(cm)

[[697 270]
 [  7  26]]


In [25]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)

In [38]:
print(mse)

0.277


In [27]:
import numpy as np

std = np.std(y_test - predictions)

In [39]:
print(std)

0.4558848538830812


In [28]:
report_dict = {
    "regression_metrics": {
        "mse": {
            "value": mse,
            "standard_deviation": std
        },
    },
}

In [32]:
import json
metrics_location = 'evaluation.json'

with open(metrics_location, 'w') as f:
        json.dump(report_dict, f)

In [34]:
print(report_dict)

{'regression_metrics': {'mse': {'value': 0.277, 'standard_deviation': 0.4558848538830812}}}
